## optimize bimax module
1, use langmuir wave dispersion relation to find the poles in k plane.

Toolkit paper:
    $k_b \approx \frac{1+ \alpha}{L_{Dc}}[\frac{1-\omega_{pT}^2/\omega^2}{3(1 + \alpha t)}]^{1/2}$

Then $\xi_b \equiv \omega/k_b v_{Tc} = \frac{\omega L_{Dc}/v_{Tc}}{1+\alpha} 
[\frac{3(1 + \alpha t)}{1-\omega_{pT}^2/\omega^2}]^2$

$\frac{\omega/\omega_{pc}}{\sqrt{2}(1+\alpha)} 
[\frac{3(1 + \alpha t)}{1- (1+\alpha)\omega_{pc}^2/\omega^2}]^2$

In [1]:
from qtn.bimax import BiMax
from qtn.util import timing, zpd, zp_sp, zpd_sp, f1, f1_sp, j0
from scipy.optimize import fsolve, root
import sympy.mpmath as mp
import numpy as np
import scipy.integrate

In [2]:
ant_len = 50      # m (monopole) 
ant_rad = 1.9e-4  # m
base_cap = 20e-12 # Farad
fbins = np.array([4000*1.0445**i for i in range(96)])

In [3]:
p = BiMax(ant_len, ant_rad, base_cap)

def z_b(wc, n, t):
    """
    the approximated expression for the pole,
    when frequency is close to plasma frequency
    
    see Meyer-Vernet & Perche (1989), the "toolkit" paper
    """
    term_1 = wc/(np.sqrt(2) * (1 + n))
    term_2 = 3 * (1 + n * t) / (1 - (1 + n) / wc**2 )
    term_2 = np.sqrt(term_2)
    return term_1 * term_2


In [4]:
    #@timing
    def d_l(z, wc, n, t):
        """
        Longitudinal dispersion tensor
        z: w/kv_Tc
        wc: w/w_pc
        n: n_h/n_c
        t: T_h/T_c
    
        """
        return 1 - (z/wc)**2 * (zpd(z) + n/t * zpd(z/mp.sqrt(t)))
    #@timing
    def d_l_sp(z, wc, n, t):
        """
        Longitudinal dispersion tensor
        z: w/kv_Tc
        wc: w/w_pc
        n: n_h/n_c
        t: T_h/T_c
        
        use scipy instead of mpmath
        """
        return 1 - (z/wc)**2 * (zpd_sp(z) + n/t * zpd_sp(z/np.sqrt(t)))

In [5]:
z, wc, n, t = 5.1, 1.02, 0, 1
d_l(z, wc, n, t)
d_l_sp(z, wc, n, t)

(-0.022803997275301136+2.2862074859912556e-09j)

In [74]:
    #@staticmethod
    @timing
    def long_interval_sp(wc, n, t):
        """
        Return the interval of integration for longitudinal impedance, which
        is of the form [0, root, inf], where root satisfies
        d_l(root, w, n, t) = 0.
        
        """
        if wc <= np.sqrt(1+n) or wc > np.sqrt(1.2 * (1+n)):
            return [0, np.inf]
        guess = z_b(wc, n, t)
        print(guess)
        try:
            # still used multiprecision function so that a complex solution is returned
            root = mp.findroot(lambda z: d_l_sp(np.complex(z), wc, n, t), guess)
            print(root)
            int_range = [0, np.abs((mp.re(root))), mp.inf]
            return int_range
        except ValueError:
            print('im here')
            return [0, mp.inf]


In [78]:
wc, n, t = 1.08, 0, 1
guess = 3.
# mp.findroot(lambda z: d_l_sp(z, wc, n, t), guess)
long_interval_sp(wc, n, t)

3.50200272372
(3.45207030162424 - 0.00720831828234414j)
long_interval_sp function took 6.571 ms


[0, mpf('3.4520703016242407'), mpf('+inf')]

In [14]:
    @timing
    def za_l_integrand(z, wc, l, n, t):
        """
        Integrand of longitudinal component of the antenna impedance.
        l: l/l_d, where l is antenna length, l_d is debye length
        a: a/l is the ratio of antenna radius and monopole length
        
        """
        kl = wc*l/mp.sqrt(2)/z
        ka = kl * p.al_ratio
        num = f1(kl) * j0(ka)**2
        denom = z**2 * BiMax.d_l(z, wc, n, t)
        return num/denom
    #@timing
    def za_l_integrand_sp(z, wc, l, n, t):
        """
        Integrand of longitudinal component of the antenna impedance.
        l: l/l_d, where l is antenna length, l_d is debye length
        a: a/l is the ratio of antenna radius and monopole length
        
        """
        kl = wc*l/np.sqrt(2)/z
        ka = kl * p.al_ratio
        num = f1_sp(kl) * j0(ka)**2
        denom = z**2 * d_l_sp(z, wc, n, t)
        return num/denom    

In [84]:
z, wc, l, n, t = 5.1, 1.2, 6.7, 0, 1
za_l_integrand(z, wc, l, n, t)
za_l_integrand_sp(z, wc, l, n, t)

za_l_integrand function took 1.156 ms


(0.0067389920152918174-4.2644969563453513e-11j)

In [122]:
    @timing
    def za_l(wc, l, n, t, tc):
        """
        Longitudinal impedance in unit of Ohms.
        wc: w/w_pc, where w_pc is core electron plasma frequency.
        tc: core electron temperature.
        """
        limits = p.long_interval(wc, n, t)
        print(limits)
        result = mp.quad(lambda z: p.za_l_integrand(z, wc, l, n, t), limits)
        return result 
    @timing
    def za_l_sp(wc, l, n, t, tc):
        """
        Longitudinal impedance in unit of Ohms.
        wc: w/w_pc, where w_pc is core electron plasma frequency.
        tc: core electron temperature.
        """
        limits = long_interval_sp(wc, n, t)
        print(limits)
        if len(limits) == 2:
            intgrl = scipy.integrate.quad(
                lambda z: za_l_integrand_sp(z, wc, l, n, t), 0, np.inf, limit=100)
            result = intgrl[0]
        if len(limits) == 3:
            sing = limits[1]
            intgrl_1 = scipy.integrate.quad(
                lambda z: za_l_integrand_sp(z, wc, l, n, t), 0, 2 * sing, points = [sing], limit=200)
            print(intgrl_1)
            intgrl_2 = scipy.integrate.quad(
                lambda z: za_l_integrand_sp(z, wc, l, n, t), 2 * sing, np.inf)
            result = intgrl_1[0] + intgrl_2[0]
        return result

In [120]:
z, wc, l, n, t, tc = 5.1, 1.08, 6.7, 0, 1, 1
za_l(wc, l, n, t, tc)

[0 mpf('3.452077827512928') mpf('+inf')]
za_l function took 835.108 ms


mpc(real='1.585257043662984', imag='-0.30579893644370593')

In [123]:
za_l_sp(wc, l, n, t, tc)

/usr/local/lib/python3.4/site-packages/scipy/integrate/quadpack.py:386: ComplexWarning: Casting complex values to real discards the imaginary part
  return _quadpack._qagpe(func,a,b,the_points,args,full_output,epsabs,epsrel,limit)
/usr/local/lib/python3.4/site-packages/scipy/integrate/quadpack.py:352: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  warnings.warn(msg, IntegrationWarning)


3.50200272372
(3.45207030162424 - 0.00720831828234414j)
long_interval_sp function took 5.982 ms
[0, mpf('3.4520703016242407'), mpf('+inf')]
(1.5775455207467586, 1.754594418879938e-05)
za_l_sp function took 321.718 ms


/usr/local/lib/python3.4/site-packages/scipy/integrate/quadpack.py:378: ComplexWarning: Casting complex values to real discards the imaginary part
  return _quadpack._qagie(func,bound,infbounds,args,full_output,epsabs,epsrel,limit)


1.5860654483435699

In [ ]:


@timing
def mp_quad(a, b):
    return mp.quad(lambda z: zpd_sp(float(z)), [a, b])

@timing
def sp_quad(a, b):
    return scipy.integrate.quad(lambda z: np.imag(zpd_sp(z)), a, b)

a, b = 0, 10
mp_quad(a, b)
sp_quad(a, b)

In [ ]:
sp_quad(0, 1)